## 🎆 [Day 12](https://adventofcode.com/2018/day/12)

In [0]:
class Plant:  
  """A class to describe a Plant in our garden"""
  
  def __init__(self, plant, R):
    """Initialize plants starting from the rightmost of the initial configuration"""
    self.has_plant = (plant == '#')
    self.has_plant_next_generation = False
    self.R = R
    self.L = None
    self.index = 0
    if R is not None:
      self.index = R.index - 1
      R.L = self
      
  def get_context(self):
    """Return the content (left left, left, plant, right, right right) of the current plant"""
    if self.L is None:
      return [False, False, self.has_plant, self.R.has_plant, self.R.R.has_plant]
    elif self.R is None:
      return [self.L.L.has_plant, self.L.has_plant, self.has_plant, False, False]
    elif self.L.L is None:
      return [False, self.L.has_plant, self.has_plant, self.R.has_plant, self.R.R.has_plant]
    elif self.R.R is None:
      return [self.L.L.has_plant, self.L.has_plant, self.has_plant, self.R.has_plant, False]
    else:
      return [self.L.L.has_plant, self.L.has_plant, self.has_plant, self.R.has_plant, self.R.R.has_plant]
      
  def match_pattern(self, pattern, target):
    """Check that the given pattern matches and apply it if it does"""
    try:
      values = []
      for check, current in zip(pattern, self.get_context()):
        if check != current:
          raise ValueError
      self.has_plant_next_generation = target
    except ValueError:
      pass
    finally:
      if self.R is not None: 
        self.R.match_pattern(pattern, target)     
    
  def grow_plants(self):
    """Update the plant based on pattern matching results at the previous generation"""
    self.has_plant = self.has_plant_next_generation
    self.has_plant_next_generation = False
    if self.R is not None:
      self.R.grow_plants()
      
  def trim_left(self):
    """Return the first grown plant on the left side of self (including self)"""
    if self.has_plant or self.L is None:
      return self
    else:
      return self.L.trim_left()
      
  def trim_right(self):
    """Return the first grown plant on the right side of self (including self)"""
    if self.has_plant or self.R is None:
      return self
    else:
      return self.R.trim_right()
      
  def lively_sum(self, offset_indices=0):
    """Return the sum of indices at positions where a plant is grown"""
    if self.R is None:
      return (self.index + offset_indices) * self.has_plant
    else:
      return (self.index + offset_indices) * self.has_plant + self.R.lively_sum(offset_indices=offset_indices)
      
  def __str__(self):
    """String representation"""
    s = '#' if self.has_plant else '.'
    return s + ('' if self.R is None else str(self.R)) 

In [0]:
def init_plants(line):
  """Initialize the plants objects and return the first (index 0) and last plant (as given by the input file)"""
  R = None
  leaf = None
  
  for plant in line[::-1]:
    p = Plant(plant, R)
    if R is None:
      leaf = p
      p.index = len(line) - 1
    R, RR = p, R
  return p, leaf

def grow_extremes(root, leaf, patterns, trim=False):
  """Grow additional plants on the Left or Right *if* there are growing at the next iteration.
  This is assuming the pattern ".... => #" is not possible
  Additionally trim the plants so that first and last are always grown plants, to detect repeating patterns more easily
  """
  next_root = None
  # Left of root
  checks = [False, False, False, root.has_plant, root.R.has_plant]
  for pattern, target in patterns:
    try:
      for check, plant in zip(pattern, checks):
        if check != plant:
          raise ValueError
      next_root = Plant(False, root)
      next_root.has_plant_next_generation = target
    except ValueError:
      continue
      
  # Left Left of root
  checks = [False, False, False, False, root.has_plant]
  for pattern, target in patterns:
    try:
      for check, plant in zip(pattern, checks):
        if check != plant:
          raise ValueError
      next_root = Plant(False, Plant(False, root) if next_root is None else next_root)
      next_root.has_plant_next_generation = target
    except ValueError:
      continue
      
  # Right of leaf
  next_leaf = None
  checks = [leaf.L.has_plant, leaf.has_plant, False, False, False]
  for pattern, target in patterns:
    try:
      for check, plant in zip(pattern, checks):
        if check != plant:
          raise ValueError
      next_leaf = Plant(False, None)
      next_leaf.L = leaf
      leaf.R = next_leaf
      next_leaf.index = leaf.index + 1 
      next_leaf.has_plant_next_generation = target
    except ValueError:
      continue
      
  # Right Right of leaf
  checks = [leaf.has_plant, False, False, False, False]
  for pattern, target in patterns:
    try:
      for check, plant in zip(pattern, checks):
        if check != plant:
          raise ValueError
      # Init left of next_leaf
      if next_leaf is not None:
        next_leaf_left = next_leaf    
      else:
        next_leaf_left = Plant(False, None)
        next_leaf_left.index = leaf.index + 1 
        next_leaf_left.L = leaf
        leaf.R = next_leaf_left
      # Init next_leaf
      next_leaf = Plant(False, None)  
      next_leaf.index = next_leaf_left.index + 1 
      next_leaf.has_plant_next_generation = target
      next_leaf.L = next_leaf_left
      next_leaf_left.R = next_leaf
    except ValueError:
      continue
      
  # Return
  root = root if next_root is None else next_root
  leaf = leaf if next_leaf is None else next_leaf
  return root, leaf

  
def trim_plants(root, leaf):
  """Update the root and leaf tho the first grown plant. Also updates the indices.
  Returns the updated nodes as well as the shift direction"""  
  root = root.trim_right()
  root.L = None  
  
  leaf = leaf.trim_left()
  leaf.R = None  
  
  return root, leaf


def parse_pattern(p):
  """Parse a pattern to booleans"""
  pattern = [(x == '#') for x in p[:5]]
  target = (p[-1] == '#')
  return pattern, target


def grow_plants(inputs, num_generations=20, trim=False, verbose=False):
  """Grow the plants and return the sum of indices where plants are grown"""
  # Init
  init, patterns = inputs
  root, leaf = init_plants(init)
  patterns = [parse_pattern(p) for p in patterns if p[-1] == '#']
  
  # Curent state
  offset_indices = 0
  current_root_index = 0
  current_leaf_index = 0
  current_plants = str(root)
  if verbose:
      print('%02d: %s' % (0, current_plants))
      
  # At each generation
  for n in range(num_generations):
    # Match patterns and Grow new plants
    for p, t in patterns:
      root.match_pattern(p, t) 
    root, leaf = grow_extremes(root, leaf, patterns, trim=trim)
    root.grow_plants()
    
    # Trim to detect periodic pattern more easily
    if trim: 
      root, leaf = trim_plants(root, leaf)
      # If stable point found, we only need to shift indices and finish
      if (current_plants == str(root)):
        if verbose:
          print('%02d: %s' % (n + 1, str(root)))
        shift_root = root.index - current_root_index
        shift_leaf = leaf.index - current_leaf_index
        assert shift_root == shift_leaf
        print('  Found repeating pattern with periodicity %d!' % shift_root)
        print('  %02d: %s' % (n + 1, str(root)))
        offset_indices = shift_root * (num_generations - n - 1)
        break
        
    # Next iteration
    current_root_index = root.index
    current_leaf_index = leaf.index
    current_plants = str(root)
    if verbose:      
      print('%02d: %s' % (n + 1, current_plants))
      
  return root.lively_sum(offset_indices=offset_indices)

In [3]:
with open("day12.txt", 'r') as f:  
  inputs = f.read().splitlines()
  inputs = ((inputs[0][15:], inputs[2:]))  
  
print('\nSum of grown plants indices on 20 generations:', grow_plants(inputs, num_generations=20, verbose=True))

00: #.####...##..#....#####.##.......##.#..###.#####.###.##.###.###.#...#...##.#.##.#...#..#.##..##.#.##
01: ####..##..#..###........#..#......##..#..#.##....###.#.####.###...#.###..##..#.#..#.######....##..#..#
02: ...#..#..###..###......######.....#..######..#....#...##...#####.######..#..##..#####....##...#..######
03: ..######..##...###..........##...###......#.###..###..#.#.......##....#.###.#.......##...#.#.###......##
04: .......#..#.#...###.........#.#...###....######...##.##..#......#.#..#####...#......#.#.##..#####.....#.#
05: ......#####..#...###.......##..#...###........##..###...###....##..#.....##.###....##..#........##...##..#
06: ..........#.###...###......#..###...###.......#....###...###...#..###....#######...#..###.......#.#..#..###
07: .........#######...###....###..###...###.....###....###...###.###..###.........##.###..###.....##..####..###
08: ...............##...###....##...###...###.....###....###...#.####...###........######...###....#......#...###
09: .......

In [4]:
print('Sum of grown plants indices on 50000000000 generations:')
print('Result:', grow_plants(inputs, num_generations=50000000000, trim=True, verbose=False))

Sum of grown plants indices on 50000000000 generations:
  Found repeating pattern with periodicity 1!
  92: ###...###...###...###...###...###...###...###...###...###...###...###...###...###...###...###...###...###...###...###...###...###....###...###
Result: 3600000002022
